# Project-2

This is project-2 which will get you familiar with Structure from Motion and a rudimentary stereo SLAM pipeline. This notebook is not meant for coding, refer to the notebooks in `questions/` for more information.

**Team Name**:

**Team Members/Rollnumbers**:

### Submission Instructions:
1. Zip the entire folder, not just this notebook, and submit on Moodle. 
2. Be sure to solve all questions that are in this notebook

This assignment will take around 10-12 hours, so be sure to split the work equally. A recommendation to split the work would be to have one person start with stereo, the other with bundle adjustment, and then both switch to completing the PnP and ICP parts. This is because question-1 and 2 are separate.

## Question 1: Stereo, PnP, and ICP

Time: ~6-7 Hours

This question has been designed to test your understanding of the concepts that create a SLAM system, from creating a front-end using Stereo Matching to a backend where you optimise to get your odometry to stitch the pointclouds obtained from the front end to build a map of the environment.

The dataset has been provided in `data/`. `img2` and `img3` correspond to the left and right camera respectively. `poses.txt` has the pose information for all frames. `calib.txt` has camera calibration information..

## Question 2: Bundle Adjustment

Time: ~2-3 Hours

This question will walk you through a simple implementation of Bundle Adjustment and run it on a a small dataset. Most of the weightage for this is in the theory part, where you have to explain the method.

# Question 1: Reading

1. Read the paper 'Building Rome in a Day' and briefly write about the fundamental idea behind the problem and solution. No need to be verbose, just write about the challenge with the task and how the pipeline is implemented (do not include details about performance/parallelization).

2. How is this task different from a SLAM problem?

Here it is generating a 3D scene with help of 2D images from different camera position. A 3D scene is reconstructed from 2D images from different perspective of camera. Whereas in visual SLAM, we calculate position of the camera with respect to its surroundings and simultaneously generate a map of the environment.

**ANS.** 1. The basic idea of this paper is to get use the already available data on the internet to create the dense 3D maps of the some cities that includes Dubrovnik, Rome and Venice, that too processing all the data in a single day. 

The major challenge here was that the pictures taken are from different types of camera with varying settings, and the pictures are in no particular order.

The whole process can be divided into step which are as follow:
1.  **Finding the correspondences** is the 1st step, for which SIFT framework was used to find similar looking features across different images. Then tracks are formed between the same point in 3D space.
As the number of images are in order of millions, therefore a technique called bag of words is applied.
In this technique images with visual words are divided into clusters, which help in efficiently matching large sets of images.

2. **SFM (structure from motion)** is performed to achieve a sparse 3D map of the object/environment.
The goal of this step is to find the camera viewpoints and 3D position of the points in the scene.
So basically we have to find the camera postions of that minimizes the sum of squares of the reprojection error (a non-linear squares problem). To achieve this 
a). First it is done on a small number of camera and points
b). Then start adding more cameras and points over time, and change the objective function accordingly.
    This process is known as bundle adjustment. After this step, we'll have our sparse 3D reconstruction,
    as 3D data can only be recovered at the correspondences which we achieved through SIFT.

3. As we have the rotation and translation information about the different camera, **MVS (multi-view stereo)** is applied to get the dense 3D map. In this paper patchbased consistency-check MVS algorith is used.
After which a depth map of each pixel is created.

# Theory

## 1. SfM pipeline (`6 mark`)

To get the context of below questions, take a look at the code above: The same questions have been asked at different places above as comments in the code.

1. `0.5 mark` **Basics** - How do we know this (`camera_ind`) information in practical setting? In other words, how do we know observations in `points_2d` belong to which camera. Explain. 
    - Ans-1 - Basics: Practically, when camera is capturing an image we know the exact location of the camera, therefore we obviously know the `camera_ind` before hand for each point.
    
    
2. `0.5 mark` **Basics** - How do we know this (`point_ind`) information in practical setting?  In other words, how do we know observations in `points_2d` belong to which 3D point. Explain.
    - Ans-2 - Basics: As we already know the poses of different images, we find the correspondence using SIFT, & with the help of triangulation to find that point in 3D. 
    
    
3. `0.5 mark` **Transformations** - `rotate()` function: Why do we use the rodriquez formula? How is this representation different from the standard 3x3 Rotation matrix, why do we use this instead?
    - Ans-3 - Transformations: Rodriques formula is used to find the rotation matrix from axis angle representation. The standard 3x3 rotation matrix represents the rotation about a fixed axis by an angle theta whereas the axis-angle representation is used to rotate all the three axis about a specific axis by an angle theta.
    
4. `0.5 mark` **Transformations** - `project()` function: In the `project()` function, would it make any difference if I do translate first, then rotate? Why/why not?
    - Ans-4 - Transformations: In the given function, the rotation is done first and then translation. If we perform inverse operation then it will be different as the perspective of camera frame changes. If we perform rotation first, then the orientation of the camera frame will be at origin. But if we perform inversly then the location of camera will be changed and then it is rotated resulting in different orientation than before.
        
        
        
5. `0.5 mark` **Jacobian** - `bundle_adjustment_sparsity()` function: m above is not "MN" (*2) unlike our lecture notes. Why is that so?
    - Ans-5 - Jacobian: In our lecture notes, the **M** is the number of images and **N** is the number of points per image. So the total number of 2-D point values comes out to be **2**(x & y coordinates) * **M** * **N** = **2MN**.
    But here in the code the number of 2-D points are given as **m**.
    
    Here, the number of points taken for different images are different unlike out lecture notes, where we took number of points per image to be constant.
    
    
6. `2 mark` **Jacobian & Parameters** - `bundle_adjustment_sparsity()` function: 
    1.  Why are we doing `n_cameras * 9` here instead of `n_cameras * 12`? Recollect: Every individual motion Jacobian was (1*)12 in our lecture notes. 
        - Ans 6.1 - Jacobian & Parameters: In the lecture notes, we used matrix representation, but in the function axis-angle representation or vector form of rotation is used.
        
    2. Ignoring the scale parameters, what was the number of unknown parameters in our lecture notes in terms of `n_cameras` and `n_points`? What is it here in the code? Is it different? If so, what is and why? [Link of notes](https://www.notion.so/Stereo-Structure-from-Motion-9fdd81e4194f4803ac9ba7552df56470).
        - Ans 6.2 - Jacobian & Parameters: In our lecture notes, for each of the cameras we have a projection matrix **$P$**, having a size of 3X4=12 `n_cameras`, but here in this function axis-angle representation of rotation is used, therefore totat no. of parameter comes out to be **3**(Rotation parameters) + **3**(Translation parameters) + **1**(focal length) + **2**(distortion parameters) = **12**. 
        
        
                  
            
7. `6 mark` **Sparsity, Residual Vector & Jacobian** - `bundle_adjustment_sparsity()` function: Explain what you understand from above 6 lines of code by coding a simple toy example yourself to illustrate how it is different from what you've learnt in class. ([Coding toy example + elaborating in words]- both are compulsory.) For the toy example, you can take something like 3 points all seen from 3 cameras. (You don't actually have to code much, just need to call the existing function) Write that toy example after this cell
    - Ans 6 - Sparsity, Residual Vector & Jacobian: 
        - The code creates a sparse matrix that defines the sparcity structure of the jacobian. The locations with non-zero values are set to 1.
        - Lets assume that the no. of unknown parameter is **n** and total no. of points in **m**, then (2m,n) will be the shape of the Jacobian matrix and the values in Jacobian will be non-zero only for the parameters which are used to predict that specific point in that specific image. The size of the residual will be (2m).

        

In [4]:
from scipy.sparse import lil_matrix
import numpy as np
n_image = 2 # this is same as number of cameras
n_points = 10
uniq_points = 4
n_variables = n_image*9 + n_points*3

# allocating camera_indices to each point randomly
pointCameraMapping = np.random.randint(n_image, size=n_points)
# allocating the pointIndices to each point randomly
pointIndices = np.random.randint(uniq_points, size=n_points)

# setting the dimentions of Jacobian matrix
jacDims = (2*n_points, n_variables)

# generating empty Jacobian array so that we can verify the implementation
jacSparcity = np.zeros(jacDims, dtype=int)
jacSparcityAlt = lil_matrix(jacDims, dtype=int)

pointIndices = np.arange(n_points)
# as there are 9 variables per image, we run the next loop 9 times
for i in range(9):
    # updating Jac for both x and y points for each point
    jacSparcity[pointIndices*2, pointCameraMapping*9+i] = 1
    jacSparcity[pointIndices*2+1, pointCameraMapping*9+i] = 1
    
    # repeting the same above steps for the alternate matrix created
    jacSparcityAlt[pointIndices*2, pointCameraMapping*9+i] = 1
    jacSparcityAlt[pointIndices*2+1, pointCameraMapping*9+i] = 1

# not updating the Jac for the 3d points for each x,y,z. Its indices will be after the camera parameters
for i in range(3):
    jacSparcity[pointIndices*2, n_image*9+ pointIndices*3 +i] = 1
    jacSparcity[pointIndices*2+1, n_image*9+ pointIndices*3 +i] = 1

    jacSparcityAlt[pointIndices*2, n_image*9+ pointIndices*3 +i] = 1
    jacSparcityAlt[pointIndices*2+1, n_image*9+ pointIndices*3 +i] = 1

print(jacSparcity)
print(jacSparcityAlt.toarray())
print("Both matrix are same:",np.all(jacSparcity==jacSparcityAlt.toarray()))


[[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0
  0 0 0 0 0 0 0 0

## 2. Initializing R,t and 3D points for SfM given 2 images (`4 mark`)

Using OpenCV functions, mention how you would initialize R,t (poses) and 3D points for SfM given 2 images and K matrix. You don't need to implement it, just mention function names with input/output arguments clearly and briefly explain what they do (You don't need to give detailed answers).

1. First step will be to load the both images for which we will use the function `cv.imread(image_name.extension)`


2. Next step is to find out the correspondences we can use ORB (oriented FAST and rotated BRIEF)
`img1Keypoints, img1Descriptors = orb.detectAndCompute(image1,None)`
`img2Keypoints, img2Descriptors = orb.detectAndCompute(image2,None)`
The above two function will detect the keypoint and compute the descriptors for both the images.


3. BFMatches (Brute force matcher) is used match the correspondences, the function used here is `matches = matcher.match(img1Descriptors, img2Descriptors)`, for which the descriptors are input, are it returns the matches as output.


4. To calculate the essential matrix which contains the information about Rotation *R* and translation *t*. The function used here is `E, mask = cv.findEssentialMat(img2Keypoints, img1Keypoints, K,'Ransac')`, which take keypoints and camera matrix as input and gives essential matrix as output which contain information about the Rotation *R*, and translation *t*.


5. To calculate *R* and *t*, `[R,t,mask] = cv.recoverPose(E, img1Keypoints, img2Keypoints, K, mask);`